# 1. Setting Up Colab on the Google VM
How to use the Colab interface on a remote Google virtual machine that is mounted with a Google Bucket.

## Initial Set Up   *(only has to be done once per account)*
(Locally)
1. `gcloud init` (if you haven’t already)
2. `gcloud compute config-ssh` (if you haven’t already, create your password)

(On VM)
0. Start VM (if it isn't already) on the Google VM page and under the connect column, go to the SSH tab and select 'Open in browser window'
1. Follow the instructions for installing gcsfuse here https://github.com/GoogleCloudPlatform/gcsfuse/blob/master/docs/installing.md
2. Install conda

>`wget http://repo.continuum.io/archive/Anaconda3-4.0.0-Linux-x86_64.sh`
>`bash Anaconda3-4.0.0-Linux-x86_64.sh`

>Note that if this gives you an error involving unpackaging the zip file, try:

>`sudo apt-get install bzip2`
>`rm -r /home/<your_username>/anaconda3`

3. `source ~/.bashrc`
4. `conda install jupyter`
5. `conda install tornado=4.5.3` (resolving depending issue)
6. `pip install jupyter_http_over_ws`
7. `pip install requests==2.4.3`
8. `jupyter serverextension enable --py jupyter_http_over_ws`
9. `conda upgrade ipykernel`

*In order to run colab (or any notebook) from a VM, we have to run the notebook from the VM, and then pipe the output of that VM on to our local machine (we'll be using port 8888). Once we do this, we can have a display run 'locally' (either through Jupyter notebook or colab).*

## Running the VM on Colab
(Locally)
1. In your terminal, run:

>`gcloud compute ssh --zone us-west1-b dfd-cpu-instance -- -L 8888:localhost:8888`

>(replace us-west1-b and dfd-cpu-instance with your VM's region and instance name respectively)

(On VM)
1. Start VM (if it isn't already) on the Google VM page and under the connect column, go to the SSH tab and select 'Open in browser window' and under the connect column, go to the SSH tab and select 'Open in browser window.' If you are already in the VM, make sure you are in your home directory.
3. `mkdir satellite && cd satellite` (we called our data directory satellite but you can call it anything you want)
4. Check if there's anything in this directory. If not, run (in the satellite folder): `gcsfuse tent-bucket .`
5. In the pop-up terminal, type:
`jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'  --port=8888 --no-browser`
6. Copy and paste link in URL it prints out (it will direct to a jupyter notebook page)
7. From here you can directly create/run notebooks, or go to the next step to connect to the colab
8. In the colab, go to the runtime option (click the down arrow), connect to 'local' and make sure the port is 8888 (or whatever you set it as in part 6)
9. Now you can run the rest of this colab! :)


# 2. Processing Digital Globe Data in to Image Files
Processing our Digital Globe Data, which exists as TIF files in to consumable chunks of images and geo-mapping the correpsonding structures in our labelling data to each output image.

In [6]:
# Install necessary imports for processing TIF files
!pip install rasterio

# Import necessary libraries
import base64
import json
import pandas as pd
from PIL import Image
from io import BytesIO
from collections import defaultdict

# import geopandas as gpd
# from geopandas import GeoDataFrame
import descartes
from shapely.geometry import Point
import numpy as np

from rasterio.io import MemoryFile
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show

import random
import glob

You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
# This is the directory of the mounted google bucket
GOOGLE_BUCKET_DIR = 'satellite/'

# NOTE: I added 'labels' directory before each file now that it is local
# and we are storing the geojson rating data here
REGION_MAPPINGS = {
    'Africa1': {
        'qc': GOOGLE_BUCKET_DIR + 'labels/africa1_qcexport_20180906.geojson',
        'raw': GOOGLE_BUCKET_DIR + 'labels/africa1_rawtags_20180906.geojson'},
    'Africa2': {
        'qc': GOOGLE_BUCKET_DIR + 'labels/africa2_qcexport_20180927.geojson',
        'raw': GOOGLE_BUCKET_DIR + 'labels/africa2_rawtags_20180927.geojson'},
    'Bangladesh': {
        'qc': GOOGLE_BUCKET_DIR + 'labels/bangladesh_qcexport_20180906.geojson',
        'raw': GOOGLE_BUCKET_DIR + 'labels/bangladesh_rawtags_20180906.geojson'},
    'WesternAsia': {
        'qc': GOOGLE_BUCKET_DIR + 'labels/westernAsia_qcexport_20180906.geojson',
        'raw': GOOGLE_BUCKET_DIR + 'labels/westernAsia_rawtags_20180906.geojson'}
}

COLORS = {'UNHCR Tent': 'r',
          'Administrative Structure': 'm',
          'Round Earthen Structure': 'c',
          'Other Tent': 'y'}

CHIP_URL_PREFIX = 'https://s3.amazonaws.com/explorationlab/chips/'

In [0]:
def get_features(filepath, region):
  """
  Featurize datasets (QC and Raw) according to the following schema from the
  stored json object. Note that we:
    1) Ignore the metadata tags and only use the 'feature tag'
    2) Expand the geography tag within the feature tag to 'latitude',
       'longitude', and 'coordinate_type' keys, in addition to the raw geojson
    3) Convert date features to pandas Timestamp and confidence score to float

  QC datasets
  id: unique identifier
  label: name for the feature type
  score: the CrowdRank confidence score of the assigned attribute, relative to other tags in the dataset. (floating point 0-1 with 0=no confidence, 1= fully confident)
  agreement: number of other taggers who placed the same tag type on this point
  chip_url: link to a .jpg image chip hosted on aws with the identified feature in the center of the image
  timestamp: Date/time in GMT of the last QC batch
  acquisition_date: Date the image was collected
  sensor: the DigitalGlobe satellite that captured the image
  catalog_id: DigitalGlobe identifier for the image strips collected by our satellites
  map_id: unique identifier for each lattice cell across the imagery

  Raw tags
  id: unique identifier
  tagger_id: unique identifier of the user
  map_view_id: not too helpful for you, but I can use this to link back to another table in the database for the map/image seen by the crowd
  type_id: database tag for the feature type
  label: name for the feature type
  timestamp: time in GMT that tag was placed
  """
  with open(filepath) as f:
    content = f.read()
  obj = json.loads(content)
  data = defaultdict(list)
  for x in obj['features']:
    data['latitude'].append(x['geometry']['coordinates'][1])
    data['longitude'].append(x['geometry']['coordinates'][0])
    data['coordinate_type'].append(x['geometry']['type'])
    for key in x['properties'].keys():
      if key == 'acquisition_date' or key == 'timestamp':
        data[key].append(pd.Timestamp(x['properties'][key]))
      elif key == 'score':
        data[key].append(float(x['properties'][key]))
      elif key == 'chip_url':
        data[key].append(x['properties'][key].replace(CHIP_URL_PREFIX, ''))
      else:
        data[key].append(x['properties'][key])
  df = pd.DataFrame.from_dict(dict(data))
  df['region'] = region
  # Clean data a little - group all 'Other Tent ...' together in an 'Other' category
  df['label'] = df['label'].apply(lambda x: 'Other Tent' if x.startswith('Other Tent') else x)
  return df

In [0]:
def lonlat2xy(lon_in, lat_in, lon1, lon2, lat1, lat2, w, h):
  """Simple function to calcualte x/y coordinates based on lon/lat coordinate"""
  x = w * (lon_in - lon1) / (lon2 - lon1)
  y = h * (lat_in - lat1) / (lat2 - lat1)
  return x, y

def xy2lonlat(x_in, y_in, lon1, lon2, lat1, lat2, w, h):
  """Simple function to calcualte lat/lon coordinates based on x/y coordinate"""
  lon = lon1 + (lon2 - lon1) * x_in / w
  lat = lat1 + (lat2 - lat1) * y_in / h
  return lon, lat

In [0]:
def get_cropped_images(tif_path, labels, xcrop, ycrop):
  """
  Output image crops and corresponding labels for a given tif file and all
  labels.
  
  tif_path: path to the tif file we want to get images out of
  labels: pandas dataframe containing the tomnod data ratings file (processed
          in get_features method above)
  xcrop: width of desired crop (geospatial)
  ycrop: height of desired crop (geospatial)
  """
  with rasterio.open(tif_path) as dataset:
    
    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()
    
    big_lon1, big_lat1 = 0, 0
    big_lon2, big_lat2 = 0, 0
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

      # Transform shapes from the dataset's own coordinate
      # reference system to CRS84 (EPSG:4326).
      geom = rasterio.warp.transform_geom(
          dataset.crs, 'EPSG:4326', geom, precision=10)

      # Get coordinates from the dataset
      [big_lon1, big_lat1] = geom['coordinates'][0][0]
      [big_lon2, big_lat2] = geom['coordinates'][0][2]
    
    w, h = dataset.profile['width'], dataset.profile['height']
    # Assign crop sizes according to the resolution
    lon_crop = xcrop/w * (big_lon2 - big_lon1)
    lat_crop = ycrop/h * (big_lat2 - big_lat1)

    # Flip dimension order to have channel last instead of first
    big_im = np.transpose(dataset.read(), (1,2,0))
  
  ims, labels_crops = [], []
  
  # Iterate through each crop grid and calculate its labels
  for x1 in range(0, w, xcrop):
    for y1 in range(0, h, ycrop):
      
      # Calculate lat/lon boundaries for crop
      lon1, lat1 = xy2lonlat(
          x1, y1, big_lon1, big_lon2, big_lat1, big_lat2, w, h)
      lon2, lat2 = lon1 + lon_crop, lat1 + lat_crop
      min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
      min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)

      # Get all structures within this range and create the label
      labels_crop = labels[(labels.latitude  > min_lat) &
                           (labels.latitude  < max_lat) & 
                           (labels.longitude > min_lon) &
                           (labels.longitude < max_lon)]
      labels_crop['tile_min_lat'] = big_lat1
      labels_crop['tile_max_lat'] = big_lat2
      labels_crop['tile_min_lon'] = big_lon1
      labels_crop['tile_max_lon'] = big_lon2
      labels_crop['min_lat'], labels_crop['max_lat'] = min_lat, max_lat
      labels_crop['min_lon'], labels_crop['max_lon'] = min_lon, max_lon
      labels_crop['x'], labels_crop['y'] = lonlat2xy(
          labels_crop.longitude, labels_crop.latitude, lon1, lon2, lat1, lat2,
          xcrop, ycrop)
      
      # Create the cropped images
      im = big_im[y1:y1+ycrop, x1:x1+xcrop, :]
      ims.append(im)
      labels_crops.append(labels_crop)
    
  return ims, labels_crops

In [0]:
def upload_crops(ims, labels_crops, tif_prefix, dest=''):
  """
  Upload an array of image and labels for crops to a filepath
  ims: array of images
  labels_crops: array of pandas dataframes containing labels w.r.t each image
  tf_prefix: tif_id to store the images under
  """
  indices = [i for i in range(len(labels_crops)) if not labels_crops[i].empty]
  if len(indices) == 0:
    return
  region = labels_crops[indices[0]].region.unique()[0]
  for i in indices:
    png_filename = '%s%s/%s_%d.png' % (dest, region, tif_prefix, i)
    print('Outputting: %s' % png_filename)
    img = ims[i]
    if img.shape[-1] == 1:
      img = np.squeeze(np.stack((img,) * 3, -1)) 
    Image.fromarray(img.astype('uint8')).save(png_filename)
    csv_filename = '%s%s/%s_%d.csv' % (dest, region, tif_prefix, i)
    labels_crops[i].to_csv(csv_filename, encoding='utf-8')

In [14]:
# For the project, we copied all the code above and then moved all the code
# below under the main function (commented out below). This allowed us to run
# a script processing on all examples remotely and detach the thread.
# Note that this can also be adapted to run on the colab (although this may take
# significantly slower as the .tif files will not be mounted on the file system)

# if __name__ == "__main__":
qc = pd.DataFrame()
for region in REGION_MAPPINGS.keys():
  if len(qc) == 0:
    qc = get_features(REGION_MAPPINGS[region]['qc'], region)
  else:
    qc = qc.append(get_features(REGION_MAPPINGS[region]['qc'], region))
all_tifs = glob.glob(GOOGLE_BUCKET_DIR + '*/*.tif')
failed_indices = []
CROP_SIZE = 500
for i in range(10):
  tif = all_tifs[i]
  print("Processing index %d, tile %s" % (i,tif))
  tif_prefix = (tif.split('/')[-1]).split('.')[0]
#   try:
  ims, labels_crops = get_cropped_images(tif, qc, CROP_SIZE, CROP_SIZE)
  upload_crops(ims, labels_crops, tif_prefix)
#   except:
#     failed_indices.append(i)
print("Failed indices:")
for idx in failed_indices:
  print(idx)

Processing index 0, tile satellite/10200100058D0600/10200100058D0600_BROWSE.tif


/home/mekmarriott/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mekmarriott/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mekmarriott/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

Outputting: Africa1/10200100058D0600_BROWSE_2.png
Outputting: Africa1/10200100058D0600_BROWSE_3.png
Outputting: Africa1/10200100058D0600_BROWSE_17.png
Outputting: Africa1/10200100058D0600_BROWSE_18.png
Processing index 1, tile satellite/10200100058D0600/10200100058D0600_R10C1.tif
Processing index 2, tile satellite/10200100058D0600/10200100058D0600_R10C2.tif
Processing index 3, tile satellite/10200100058D0600/10200100058D0600_R10C3.tif
Processing index 4, tile satellite/10200100058D0600/10200100058D0600_R11C1.tif
Processing index 5, tile satellite/10200100058D0600/10200100058D0600_R11C2.tif
Processing index 6, tile satellite/10200100058D0600/10200100058D0600_R11C3.tif
Processing index 7, tile satellite/10200100058D0600/10200100058D0600_R12C1.tif
Processing index 8, tile satellite/10200100058D0600/10200100058D0600_R12C2.tif
Processing index 9, tile satellite/10200100058D0600/10200100058D0600_R12C3.tif
Failed indices:
